In [1]:
import argparse
import sys, os
sys.path.append(os.path.join(sys.path[0], '/home/ibespalov/unsupervised_pattern_segmentation/'))
sys.path.append(os.path.join(sys.path[0], '/home/ibespalov/unsupervised_pattern_segmentation/gans_pytorch/'))
sys.path.append(os.path.join(sys.path[0], '/home/ibespalov/unsupervised_pattern_segmentation/gans_pytorch/stylegan2'))
sys.path.append(os.path.join(sys.path[0], '/home/ibespalov/unsupervised_pattern_segmentation/gans_pytorch/gan/'))

from gan.gan_model import cont_style_munit_enc
from models.munit.enc_dec import MunitEncoder
from parameters.dataset import DatasetParameters
from parameters.deformation import DeformationParameters
from parameters.gan import GanParameters, MunitParameters
import albumentations
from albumentations.pytorch.transforms import ToTensor as AlbToTensor
from dataset.cardio_dataset import ImageMeasureDataset
from torch.utils import data
from transforms_utils.transforms import MaskToMeasure
import numpy as np
from transforms_utils.transforms import MeasureToMask, ToNumpy, NumpyBatch, ToTensor, MaskToMeasure, ResizeMask
from dataset.lazy_loader import LazyLoader, W300DatasetLoader, CelebaWithKeyPoints, Celeba
from dataset.toheatmap import ToHeatMap, heatmap_to_measure
from modules.hg import hg2, final_preds_untransformed, hg8, hg4, HG_softmax2020
from modules.linear_ot import SOT, PairwiseDistance
from dataset.probmeasure import ProbabilityMeasureFabric, ProbabilityMeasure, UniformMeasure2DFactory, \
UniformMeasure2D01
from torch import nn

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [5]:
from dataset.cardio_dataset import ImageMeasureDataset
from dataset.probmeasure import ProbabilityMeasure, ProbabilityMeasureFabric
from geomloss import SamplesLoss
from parameters.path import Paths

In [11]:
batch_size = 32
barycenter: UniformMeasure2D01 = UniformMeasure2DFactory.load(f"{Paths.default.models()}/face_barycenter_68").cuda().batch_repeat(batch_size)

In [8]:
def handmadew1(m1,m2):
    lambd = 0.002
    with torch.no_grad():
        P = SOT(200, lambd).forward(m1, m2)
        M = PairwiseDistance()(m1.coord, m2.coord).sqrt()
    return (M * P).sum(dim=(1,2)) / 2 # (2 * m1.coord.shape[1])


In [14]:
import torch
sum_loss = 0
for i, batch in enumerate(LazyLoader.w300().test_loader):
    landmarks = batch["meta"]["keypts_normalized"].cuda()
    landmarks[landmarks > 1] = 0.99999
    pred_measure = barycenter.slice(0, landmarks.shape[0])
    target = UniformMeasure2D01(torch.clamp(landmarks, max=1))
    eye_dist = landmarks[:, 45] - landmarks[:, 36]
    eye_dist = eye_dist.pow(2).sum(dim=1).sqrt()
    sum_loss += (handmadew1(pred_measure, target) / eye_dist).sum().item()
print(sum_loss / len(LazyLoader.w300().test_dataset))

0.1533396379002985
